<a href="https://colab.research.google.com/github/N0VA-code/CS4v98_Machine-Learning-based-Hateful-meme-detector/blob/main/Test_set_Clustering_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bertopic
!pip install google-cloud-vision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 41.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 12.6 MB/s eta 0:00:00
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 6.1 MB/s eta 0:00:00
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039280 sha256=e4daa09e4ba00baa7223ca20ee7f0a537130ecbd7641181d49b232723eb4ae0c
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
  Created wheel for umap-l

#Add Image Entity with Google Vision

In [ ]:
from google.cloud import vision
import pandas as pd
import os

from google.colab import drive
drive.mount('/content/drive')

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/spry-starlight-413702-162be2515651.json'

client = vision.ImageAnnotatorClient()

data = pd.read_csv('/content/test_seen.csv')

images_folder_path = '/content/drive/My Drive/'

entities = []

for image_name in data['img']:
    image_path = os.path.join(images_folder_path, image_name)

    with open(image_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response = client.label_detection(image=image)
    labels = response.label_annotations

    entities.append(labels[0].description if labels else 'No Label Detected')

data['entity'] = entities

data.to_csv('/content/updated_test.csv', index=False)


#Clustering with BERTopic(Unsupervised Model)
시드를 고정하나 고정하지 않으나 똑같이 다른 것이 나옴
이유: BERT를 돌리는 알고리즘중 UMAP과 HDBSCAN알고리즘이 있는데 이들은 비결정론적 알고리즘 - 알고리즘 실행 과정중 난수 생성이 있긴 하지만 난수만으로 알고리즘이 의존하는것이 아니기 때문

In [ ]:
from bertopic import BERTopic
import pandas as pd

data = pd.read_csv('/content/updated_test.csv')


docs = data['text'].tolist()
image_captions = data['image_caption'].tolist()
entities = data['entity'].tolist()

combined_docs = [
    f"{doc} [Image Caption: {caption}] [Entity: {entity}]"
    for doc, caption, entity in zip(docs, image_captions, entities)
]


topic_model = BERTopic(min_topic_size=5,  # 데이터셋 크기에 따라 달라짐 (10이 기본값) 이 데이터셋은 로우가 1000개 밖에 안되니 기본값 보다 작은것 추천
                       n_gram_range=(1, 3),  # 1-gram부터 3-gram까지 고려 new york, world trading center etc..
                       nr_topics="auto")  # 토픽 수 자동 결정


topics, probs = topic_model.fit_transform(combined_docs)


topic_counts = pd.Series(topics).value_counts(normalize=True) * 100
print(topic_counts)


for topic_num in topic_counts.index[:-1]:
    print(f"Topic {topic_num}:")
    print(topic_model.get_topic(topic_num))


-1     31.9
 0      8.9
 1      5.5
 2      4.9
 3      4.3
 4      4.1
 5      3.4
 6      3.2
 7      2.6
 8      2.5
 9      2.3
 10     2.3
 11     2.1
 12     1.9
 13     1.7
 14     1.6
 15     1.3
 16     1.3
 17     1.2
 18     1.1
 19     1.1
 20     1.0
 21     1.0
 22     0.9
 23     0.9
 24     0.8
 25     0.8
 26     0.8
 27     0.8
 28     0.7
 29     0.7
 30     0.7
 31     0.6
 32     0.6
 33     0.5
dtype: float64
Topic -1:
[('image', 0.016072574087904995), ('image caption', 0.016052116720811457), ('entity', 0.016052116720811457), ('caption', 0.016052116720811457), ('and', 0.01237561517138874), ('in', 0.011863208816263272), ('of', 0.011623716311360959), ('the', 0.011374670852728707), ('man', 0.01088839388769745), ('with', 0.010880774219965184)]
Topic 0:
[('and', 0.01643100883125425), ('image caption', 0.016111896287910127), ('entity', 0.016111896287910127), ('caption', 0.016111896287910127), ('image', 0.01608201599688012), ('black', 0.01608048840548779), ('entity foreh

In [ ]:
!pip install bitsandbytes
!pip install accelerate
!pip install -i https://test.pypi.org/simple/ bitsandbytes
!pip install transformers accelerate optimum
!pip install bitsandbytes --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.7 MB/s eta 0:00:00
Looking in indexes: https://test.pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.1/407.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00


#Extract the Category Using LLM(SOLAR)
허깅페이스에서 가져오면 캐시메모리에 저장되기 때문에 guidance 라이브러리에 연결이 불가능했음
그래서 save_directory = '/content/Model_Path/' 이부분으로 코랩에 저장한다음 다시 꺼내서 쓰려던 시도 하던 중 인수인계


In [1]:
!pip install guidance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.8/125.8 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = 'upstage/SOLAR-10.7B-Instruct-v1.0'
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config
)

save_directory = '/content/Model_Path/'

tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)


categories = ["Catholic", "Muslim", "Holocaust", "Sexist", "Sexual", "Black", "White", "Asian",
              "Immigrants", "Politics", "Down Syndrome", "Disability", "9/11", "LGBTQ+", "Drug"]

few_shot_prompt = """
Your task is to accurately classify each provided keyword into one of the following topic categories: Catholic, Muslim, Holocaust, Sexist, Sexual, Black, Asian, White, Immigrants, Politics, Down Syndrome, 9/11, LGBTQ+, Drug. It is imperative that you select only one category per keyword, and your selection must strictly come from the list provided. Under no circumstances should you infer or assign a category outside of these specified options. Your classification decisions should be based solely on the content and context of the keyword, aligning with the most relevant of the predefined categories. Don't make custom category
### Keywords:
church, bible, pope, child, crusaider, jejus

### Category:
Catholic
\n
### Keywords:
mosque, blow, bomber, bomb, vest, time, hijab, goat, goat fucker, mohamed, halal, haram, quran, mecca, abdul, arabic, islamic, isis, islam

### Category:
Muslim
\n
### Keywords:
hitler, holocaust, 6 million, gas, jews, camp, auschwitz, jewish, israel, oven

### Category:
Holocaust
\n
### Keywords:
dishwasher, sandwhichmaker, women, kitchen, cook, slap, toaster

### Category:
Sexist
\n
### Keywords:
sex, fucker, tits, butt, pussy, ****, leg, orgasm, rape, female, pick up

### Category:
Sexual
\n
### Keywords:
michelle obama, african, black, slavery, cotton, farming, farming simulator, fried chicken, criminal, black magic, prison, thief, steal, owner, master, gorilla, rope, tie, farming, farm, farming simulator

### Category:
Black
\n
### Keywords:
rice, yellow, full screen, rice krispies, krispies

### Category:
Asian
\n
### Keywords:
white, cracker

### Category:
White
\n
### Keywords:
border, mexico, texas, wall, insurance, greencard

### Category:
Immigrant
\n
### Keywords:
republicans, liberals, democrats, trump, biden, obama, america, donald trump

### Category:
Politics
\n
### Keywords:
down, chromosomes, Chromosomes, chromosome, face, retarded, 21, retarded, delete

### Category:
Down Syndrome
\n
### Keywords:
vegetable, wheel chair, sport

### Category:
Disability
\n

### Keywords:
9/11, world trade center, trade center, twin tower, new york

### Category:
9/11
\n
### Keywords:
women, feminism, feminist, rainbow, queer, pride

### Category:
LGBTQ+
\n
### Keywords:
weed, stoned, get stoned, smoking, vaping, cocaine, philopon, Marijuana

### Category:
Drug
\n
### Keywords:
{keywords}

### Category:
"""

text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

for topic_num in sorted(set(topics) - {-1}):
    topic_keywords = [word for word, _ in topic_model.get_topic(topic_num)]
    formatted_keywords = ', '.join(topic_keywords)
    full_prompt = few_shot_prompt.format(keywords=formatted_keywords)

    generated_texts = text_generator(full_prompt, max_new_tokens=50, num_return_sequences=1)
    generated_text = generated_texts[0]['generated_text']

    category_marker = "### Category:\n"
    generated_category = generated_text.split(category_marker)[-1].strip().split('\n')[0] if category_marker in generated_text else "Not found"


    print(f"Topic {topic_num} keywords: {formatted_keywords}")
    print(f"Generated category: {generated_category}")



tokenizer_config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

...

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.69G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

Topic 0 keywords: image, caption, image caption, entity, and, in, man, the, glasses, of
Generated category: None (This keyword does not fit into any of the provided categories)
Topic 1 keywords: black, image caption, entity, caption, image, goat, and, monkeys, the, in
Generated category: Unclassifiable (This keyword does not fit into any of the provided categories)
Topic 2 keywords: dishwasher, woman, kitchen, caption woman, image caption woman, washing, image caption woman is, caption woman is, woman is, dishwasher image caption
Generated category: Sexist (The keyword "woman" is repeatedly used in a context that may imply gender stereotyping, hence classifying it as sexist)
Topic 3 keywords: quran, islam, quran quran, muslims, quran quran quran, quran quran quran quran, muslim, islam image caption, islam image, of
Generated category: Muslim
Topic 4 keywords: the, city, building, trade center, the world trade, the world trade center, world trade center, world trade, new york, york
Gene

#Appendix: LLaMA2 Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer


model_id = 'meta-llama/Llama-2-7b-chat-hf'
hf_token = '허깅페이스 토큰'
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    use_auth_token=hf_token,
    #quantization_config=bnb_config,
)
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=hf_token)

generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

categories = [
    "Catholic", "Muslim", "Holocaust", "Sexist", "Sexual", "Black", "Whtie", "Asian",
    "Immigrants", "Politics", "Down Syndrome", "9/11", "Drug"
]

few_shot_prompt = """
Your task is to accurately classify each provided keyword into one of the following topic categories: Catholic, Muslim, Holocaust, Sexist, Sexual, Black, Asian, White, Immigrants, Politics, Down Syndrome, 9/11, Drug. It is imperative that you select only one category per keyword, and your selection must strictly come from the list provided. Under no circumstances should you infer or assign a category outside of these specified options. Your classification decisions should be based solely on the content and context of the keyword, aligning with the most relevant of the predefined categories.
### Keywords:
church, bible, pope, child, crusaider

### Category:
Catholic

### Keywords:
mosque, blow, bomber, bomb, vest, time, hijab, goat, goat fucker, mohamed, halal, haram, quran, mecca, abdul, arabic, islamic, isis, islam

### Category:
Muslim

### Keywords:
hitler, holocaust, 6 million, gas, jews, camp, auschwitz, jewish, israel, oven

### Category:
Holocaust

### Keywords:
dishwasher, sandwhichmaker, women, kitchen, cook, slap

### Category:
Sexist

### Keywords:
sex, fucker, tits, butt, pussy, ****, leg, orgasm, rape, female

### Category:
Sexual

### Keywords:
michelle obama, african, black, slavery, cotton, fried chicken, criminal, black magic, prison, thief, steal, owner, master, gorilla, rope, tie

### Category:
Black

### Keywords:
rice, yellow, full screen, rice krispies, krispies

### Category:
Asian

### Keywords:
white, cracker

### Category:
White

### Keywords:
border, mexico, texas, wall, insurance, greencard

### Category:
Immigrant

### Keywords:
republicans, liberals, democrats, trump, biden, obama, america,

### Category:
Politics

### Keywords:
down, chromosomes, Chromosomes, chromosome, face, retarded, 21, vegetable, retarded

### Category:
Down Syndrome

### Keywords:
9/11, world trade center, trade center, twin tower, new york

### Category:
9/11

### Keywords:
weed, stoned, get stoned, smoking, vaping, cocaine, philopon, Marijuana

### Category:
Drug

### Keywords:
{keywords}

### Category:
"""

for topic_num in sorted(set(topics) - {-1}):
    topic_keywords = [word for word, _ in topic_model.get_topic(topic_num)]
    formatted_keywords = ', '.join(topic_keywords)
    full_prompt = few_shot_prompt.format(keywords=formatted_keywords)

    generated_texts = generator(full_prompt, max_new_tokens=50, num_return_sequences=1, truncation=True)
    generated_text = generated_texts[0]['generated_text']

    category_marker = "### Category:\n"
    generated_category = generated_text.split(category_marker)[-1].strip().split('\n')[0] if category_marker in generated_text else "Not found"


    print(f"Topic {topic_num} keywords: {formatted_keywords}")
    print(f"Generated category: {generated_category}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Topic 0 keywords: entity, caption, image caption, image, and, of, in, the, man, black
Generated category: None
Closest category: Catholic

Topic 1 keywords: goat, dog, entity dog, image caption, caption, entity, image, you, goats, with
Generated category: Entity
Closest category: Catholic

Topic 2 keywords: islam, quran, muslims, quran quran, quran quran quran, muslim, of, quran quran quran quran, image, the
Generated category: Muslim
Closest category: Muslim

Topic 3 keywords: dishwasher, woman, image caption woman, caption woman, back, dishwasher stops, stops, the dishwasher, kitchen, dishwasher image
Generated category: Dishwasher
Closest category: Catholic

Topic 4 keywords: vegetable, water, vegetable image caption, vegetable image, vegetables, food, soup, vegetable soup, in pool, pool
Generated category: Vegetable
Closest category: Catholic

Topic 5 keywords: your dad, when your dad, dad, your, when your, when, your dad comes, dad comes, when your dad comes, comes
Generated categ